Valuing Actions by Estimating Probabilities

In [1]:
import pandas as pd
import numpy as np
import pandera as pa
from pandera.typing import Series, DataFrame
from typing import Any, List
import joblib

import sys
sys.path.append("/Users/ciaran/Documents/Projects/AFL/git-repositories/afl-player-ratings/")
sys.path.append("/Users/ciaran/Documents/Projects/AFL/git-repositories/expected-score-model/")

from config import chain_file_path
from exp_vaep.domain.preprocessing.preprocessing import *

import os
import warnings
warnings.filterwarnings('ignore')
pd.options.display.max_rows = 999
pd.options.display.max_columns = 999

Load Chain Data

In [2]:
chains = pd.read_csv(chain_file_path)
chains.tail()

,Chain_Number,Initial_State,Final_State,Order,Quarter,Quarter_Duration_Chain_Start,Quarter_Duration,Team_Chain,Team,Player,AFL_API_Player_ID,Description,x,y,Disposal,Shot_At_Goal,Behind_Detail,Venue_Width,Venue_Length,Home_Team,Away_Team,Home_Team_Direction_Q1,Match_ID,Round_ID,Year,Season
1169201,252,ballUp,turnover,1984.0,4,1937,1942.0,St Kilda,St Kilda,Dan Butler,Dan_Butler,Kick,-65.0,14.0,clanger,NaN,NaN,129,160,St Kilda,North Melbourne,left,202319_StKilda_NorthMelbourne,202319,NaN,2023.0
1169202,252,ballUp,turnover,1985.0,4,1937,1945.0,St Kilda,St Kilda,Dan Butler,Dan_Butler,Out On Full After Kick,-69.0,34.0,NaN,NaN,NaN,129,160,St Kilda,North Melbourne,left,202319_StKilda_NorthMelbourne,202319,NaN,2023.0
1169203,253,possGain,behind,1986.0,4,1949,1949.0,North Melbourne,North Melbourne,Luke Davies-Uniacke,Luke_Davies-Uniacke,OOF Kick In,67.0,-34.0,NaN,NaN,NaN,129,160,St Kilda,North Melbourne,left,202319_StKilda_NorthMelbourne,202319,NaN,2023.0
1169204,253,possGain,behind,1987.0,4,1949,1976.0,North Melbourne,North Melbourne,Luke Davies-Uniacke,Luke_Davies-Uniacke,Kick,67.0,-34.0,ineffective,True,NaN,129,160,St Kilda,North Melbourne,left,202319_StKilda_NorthMelbourne,202319,NaN,2023.0
1169205,253,possGain,behind,1988.0,4,1949,1978.0,North Melbourne,North Melbourne,Luke Davies-Uniacke,Luke_Davies-Uniacke,Behind,67.0,-34.0,NaN,NaN,missLeft,129,160,St Kilda,North Melbourne,left,202319_StKilda_NorthMelbourne,202319,NaN,2023.0


Load Expected Score Data

In [3]:
expected_score_version = 4
model_location = '/Users/ciaran/Documents/Projects/AFL/git-repositories/expected-score-model/model_outputs/models/'
expected_goal_set_version = 7
expected_behind_set_version = 5
expected_miss_set_version = 4

expected_goal_open_version = 7
expected_behind_open_version = 6
expected_miss_open_version = 5

In [4]:
expected_goal_set_model = joblib.load(model_location+"expected_goal_set_v"+str(expected_goal_set_version)+".joblib")
expected_behind_set_model = joblib.load(model_location+"expected_behind_set_v"+str(expected_behind_set_version)+".joblib")
expected_miss_set_model = joblib.load(model_location+"expected_miss_set_v"+str(expected_miss_set_version)+".joblib")

expected_goal_open_model = joblib.load(model_location+"expected_goal_open_v"+str(expected_goal_open_version)+".joblib")
expected_behind_open_model = joblib.load(model_location+"expected_behind_open_v"+str(expected_behind_open_version)+".joblib")
expected_miss_open_model = joblib.load(model_location+"expected_miss_open_v"+str(expected_miss_open_version)+".joblib")

In [5]:
goal_set_features = expected_goal_set_model.xgb_model.get_booster().feature_names
behind_set_features = expected_behind_set_model.xgb_model.get_booster().feature_names
miss_set_features = expected_miss_set_model.xgb_model.get_booster().feature_names

goal_open_features = expected_goal_open_model.xgb_model.get_booster().feature_names
behind_open_features = expected_behind_open_model.xgb_model.get_booster().feature_names
miss_open_features = expected_miss_open_model.xgb_model.get_booster().feature_names

In [6]:
chains[['ballUp', 'centreBounce', 'kickIn', 'possGain', 'throwIn']] = pd.get_dummies(chains['Initial_State'])

In [7]:
chains['Event_Type1'] = chains['Description'].shift(1)
shots = chains[chains['Shot_At_Goal'] == True]
shots['Set_Shot'] = shots['Event_Type1'].apply(lambda x: ("Mark" in x) or ("Free" in x))
set_shots = shots[shots['Set_Shot']]
open_shots = shots[~shots['Set_Shot']]

In [8]:
set_shots = expected_score_feature_engineering(set_shots)
open_shots = expected_score_feature_engineering(open_shots)

In [9]:
set_shots['xGoals'] = expected_goal_set_model.predict_proba(set_shots[goal_set_features], calibrate=True)
set_shots['xBehinds'] = expected_behind_set_model.predict_proba(set_shots[behind_set_features], calibrate=True)
set_shots['xMiss'] = expected_miss_set_model.predict_proba(set_shots[miss_set_features], calibrate=True)

open_shots['xGoals'] = expected_goal_open_model.predict_proba(open_shots[goal_open_features], calibrate=True)
open_shots['xBehinds'] = expected_behind_open_model.predict_proba(open_shots[behind_open_features], calibrate=True)
open_shots['xMiss'] = expected_miss_open_model.predict_proba(open_shots[miss_open_features], calibrate=True)

In [10]:
exp_shots = pd.concat([set_shots, open_shots], axis=0)
exp_shots = exp_shots.sort_values(by = ['Match_ID', "Chain_Number", "Order"])
shots = shots.merge(exp_shots[['Chain_Number', 'Order', 'Match_ID', 'xGoals', 'xBehinds', 'xMiss']], how = "left", on = ['Chain_Number', 'Order', 'Match_ID'])

In [11]:
shots['xGoals_normalised'] = shots['xGoals'] / (shots['xGoals'] + shots['xBehinds'] + shots['xMiss'])
shots['xBehinds_normalised'] = shots['xBehinds'] / (shots['xGoals'] + shots['xBehinds'] + shots['xMiss'])
shots['xMiss_normalised'] = shots['xMiss'] / (shots['xGoals'] + shots['xBehinds'] + shots['xMiss'])
shots['xScore'] = shots['xGoals_normalised']*6 + shots['xBehinds_normalised']

In [12]:
chains = chains.merge(shots[['Match_ID', 'Chain_Number', 'Order', 'xScore', 'xGoals_normalised']], how="left", on = ['Match_ID', 'Chain_Number', 'Order'])
chains[['xScore', 'xGoals_normalised']] = chains[['xScore', 'xGoals_normalised']].fillna(0)
chains.head()

,Chain_Number,Initial_State,Final_State,Order,Quarter,Quarter_Duration_Chain_Start,Quarter_Duration,Team_Chain,Team,Player,AFL_API_Player_ID,Description,x,y,Disposal,Shot_At_Goal,Behind_Detail,Venue_Width,Venue_Length,Home_Team,Away_Team,Home_Team_Direction_Q1,Match_ID,Round_ID,Year,Season,ballUp,centreBounce,kickIn,possGain,throwIn,Event_Type1,xScore,xGoals_normalised
0,1,centreBounce,goal,1.0,1,13,13.0,Brisbane Lions,NaN,NaN,NaN,Centre Bounce,0.0,0.0,NaN,NaN,NaN,138,156,Brisbane Lions,Sydney,right,202101_BrisbaneLions_Sydney,202101,2021.0,NaN,0,1,0,0,0,None,0.0,0.0
1,1,centreBounce,goal,2.0,1,13,24.0,Brisbane Lions,Brisbane Lions,Dayne Zorko,Dayne_Zorko,Hard Ball Get,8.0,-5.0,NaN,NaN,NaN,138,156,Brisbane Lions,Sydney,right,202101_BrisbaneLions_Sydney,202101,2021.0,NaN,0,1,0,0,0,Centre Bounce,0.0,0.0
2,1,centreBounce,goal,3.0,1,13,24.0,Brisbane Lions,Brisbane Lions,Dayne Zorko,Dayne_Zorko,Handball,9.0,-6.0,ineffective,NaN,NaN,138,156,Brisbane Lions,Sydney,right,202101_BrisbaneLions_Sydney,202101,2021.0,NaN,0,1,0,0,0,Hard Ball Get,0.0,0.0
3,1,centreBounce,goal,4.0,1,13,28.0,Brisbane Lions,Sydney,Oliver Florent,Oliver_Florent,Loose Ball Get,11.0,-7.0,NaN,NaN,NaN,138,156,Brisbane Lions,Sydney,right,202101_BrisbaneLions_Sydney,202101,2021.0,NaN,0,1,0,0,0,Handball,0.0,0.0
4,1,centreBounce,goal,5.0,1,13,29.0,Brisbane Lions,Sydney,Oliver Florent,Oliver_Florent,Handball,12.0,-5.0,effective,NaN,NaN,138,156,Brisbane Lions,Sydney,right,202101_BrisbaneLions_Sydney,202101,2021.0,NaN,0,1,0,0,0,Loose Ball Get,0.0,0.0


Convert to Modelling Data

In [13]:
schema_chains = convert_chains_to_schema(chains)
schema_chains.head()

,match_id,quarter,quarter_seconds,overall_seconds,team,player,start_x,start_y,end_x,end_y,action_type,outcome_type,xScore
1,202101_BrisbaneLions_Sydney,1,24.0,24.0,Brisbane Lions,Dayne Zorko,8.0,-5.0,9.0,-6.0,Hard Ball Get,effective,0.0
2,202101_BrisbaneLions_Sydney,1,24.0,24.0,Brisbane Lions,Dayne Zorko,9.0,-6.0,-11.0,7.0,Handball,ineffective,0.0
3,202101_BrisbaneLions_Sydney,1,28.0,28.0,Sydney,Oliver Florent,-11.0,7.0,-12.0,5.0,Loose Ball Get,effective,0.0
4,202101_BrisbaneLions_Sydney,1,29.0,29.0,Sydney,Oliver Florent,-12.0,5.0,-14.0,2.0,Handball,effective,0.0
5,202101_BrisbaneLions_Sydney,1,30.0,30.0,Sydney,George Hewett,-14.0,2.0,-22.0,2.0,Carry,effective,0.0


In [14]:
gamestate_features = create_gamestate_features(schema_chains)
gamestate_features.tail()

,type_Kick_a0,type_Handball_a0,type_Carry_a0,type_Uncontested Mark_a0,type_Contested Mark_a0,type_Loose Ball Get_a0,type_Hard Ball Get_a0,type_Spoil_a0,type_Gather_a0,type_Free For_a0,type_Knock On_a0,type_Shot_a0,type_Tackle_a0,type_Error_a0,outcome_effective_a0,outcome_ineffective_a0,outcome_clanger_a0,type_Kick_outcome_effective_a0,type_Kick_outcome_ineffective_a0,type_Kick_outcome_clanger_a0,type_Handball_outcome_effective_a0,type_Handball_outcome_ineffective_a0,type_Handball_outcome_clanger_a0,type_Carry_outcome_effective_a0,type_Carry_outcome_ineffective_a0,type_Carry_outcome_clanger_a0,type_Uncontested Mark_outcome_effective_a0,type_Uncontested Mark_outcome_ineffective_a0,type_Uncontested Mark_outcome_clanger_a0,type_Contested Mark_outcome_effective_a0,type_Contested Mark_outcome_ineffective_a0,type_Contested Mark_outcome_clanger_a0,type_Loose Ball Get_outcome_effective_a0,type_Loose Ball Get_outcome_ineffective_a0,type_Loose Ball Get_outcome_clanger_a0,type_Hard Ball Get_outcome_effective_a0,type_Hard Ball Get_outcome_ineffective_a0,type_Hard Ball Get_outcome_clanger_a0,type_Spoil_outcome_effective_a0,type_Spoil_outcome_ineffective_a0,type_Spoil_outcome_clanger_a0,type_Gather_outcome_effective_a0,type_Gather_outcome_ineffective_a0,type_Gather_outcome_clanger_a0,type_Free For_outcome_effective_a0,type_Free For_outcome_ineffective_a0,type_Free For_outcome_clanger_a0,type_Knock On_outcome_effective_a0,type_Knock On_outcome_ineffective_a0,type_Knock On_outcome_clanger_a0,type_Shot_outcome_effective_a0,type_Shot_outcome_ineffective_a0,type_Shot_outcome_clanger_a0,type_Tackle_outcome_effective_a0,type_Tackle_outcome_ineffective_a0,type_Tackle_outcome_clanger_a0,type_Error_outcome_effective_a0,type_Error_outcome_ineffective_a0,type_Error_outcome_clanger_a0,quarter_a0,quarter_seconds_a0,overall_seconds_a0,start_x_a0,start_y_a0,end_x_a0,end_y_a0,dx_a0,dy_a0,movement_a0,type_Kick_a1,type_Handball_a1,type_Carry_a1,type_Uncontested Mark_a1,type_Contested Mark_a1,type_Loose Ball Get_a1,type_Hard Ball Get_a1,type_Spoil_a1,type_Gather_a1,type_Free For_a1,type_Knock On_a1,type_Shot_a1,type_Tackle_a1,type_Error_a1,outcome_effective_a1,outcome_ineffective_a1,outcome_clanger_a1,type_Kick_outcome_effective_a1,type_Kick_outcome_ineffective_a1,type_Kick_outcome_clanger_a1,type_Handball_outcome_effective_a1,type_Handball_outcome_ineffective_a1,type_Handball_outcome_clanger_a1,type_Carry_outcome_effective_a1,type_Carry_outcome_ineffective_a1,type_Carry_outcome_clanger_a1,type_Uncontested Mark_outcome_effective_a1,type_Uncontested Mark_outcome_ineffective_a1,type_Uncontested Mark_outcome_clanger_a1,type_Contested Mark_outcome_effective_a1,type_Contested Mark_outcome_ineffective_a1,type_Contested Mark_outcome_clanger_a1,type_Loose Ball Get_outcome_effective_a1,type_Loose Ball Get_outcome_ineffective_a1,type_Loose Ball Get_outcome_clanger_a1,type_Hard Ball Get_outcome_effective_a1,type_Hard Ball Get_outcome_ineffective_a1,type_Hard Ball Get_outcome_clanger_a1,type_Spoil_outcome_effective_a1,type_Spoil_outcome_ineffective_a1,type_Spoil_outcome_clanger_a1,type_Gather_outcome_effective_a1,type_Gather_outcome_ineffective_a1,type_Gather_outcome_clanger_a1,type_Free For_outcome_effective_a1,type_Free For_outcome_ineffective_a1,type_Free For_outcome_clanger_a1,type_Knock On_outcome_effective_a1,type_Knock On_outcome_ineffective_a1,type_Knock On_outcome_clanger_a1,type_Shot_outcome_effective_a1,type_Shot_outcome_ineffective_a1,type_Shot_outcome_clanger_a1,type_Tackle_outcome_effective_a1,type_Tackle_outcome_ineffective_a1,type_Tackle_outcome_clanger_a1,type_Error_outcome_effective_a1,type_Error_outcome_ineffective_a1,type_Error_outcome_clanger_a1,quarter_a1,quarter_seconds_a1,overall_seconds_a1,start_x_a1,start_y_a1,end_x_a1,end_y_a1,dx_a1,dy_a1,movement_a1,type_Kick_a2,type_Handball_a2,type_Carry_a2,type_Uncontested Mark_a2,type_Contested Mark_a2,type_Loose Ball Get_a2,type_Hard Ball Get_a2,type_Spoil_a2,type_Gather_a2,type_Free For_a2,type_Knock On_a2

In [15]:
gamestate_labels = create_gamestate_labels(schema_chains)
gamestate_labels.head(10)

,exp_scores,exp_concedes
1,1.727293,0.000000
2,1.727293,0.000000
3,0.000000,1.727293
4,0.000000,1.727293
5,0.000000,1.727293
6,0.000000,1.727293
7,1.727293,0.000000
8,1.727293,0.000000
9,1.727293,0.000000
10,1.727293,0.000000


In [16]:
exp_vaep_modelling_data = pd.concat([schema_chains, gamestate_features, gamestate_labels], axis=1)

In [17]:
exp_vaep_modelling_data[['exp_scores', 'exp_concedes']].describe()

,exp_scores,exp_concedes
count,947887.000000,947887.000000
mean,0.801864,0.164472
std,1.517425,0.746967
min,0.000000,0.000000
25%,0.000000,0.000000
50%,0.000000,0.000000
75%,0.000000,0.000000
max,5.740831,5.739078


In [18]:
exp_vaep_modelling_data = get_stratified_train_test_val_columns(exp_vaep_modelling_data, response="exp_scores")
exp_vaep_modelling_data = get_stratified_train_test_val_columns(exp_vaep_modelling_data, response="exp_concedes")

In [19]:
exp_vaep_modelling_data[exp_vaep_modelling_data['exp_scoresTrainingSet']]['exp_scores'].mean(), exp_vaep_modelling_data[exp_vaep_modelling_data['exp_scoresTestSet']]['exp_scores'].mean(), exp_vaep_modelling_data[exp_vaep_modelling_data['exp_scoresValidationSet']]['exp_scores'].mean()

(0.8008729238517601, 0.8034401503594883, 0.8038570007501825)

In [20]:
exp_vaep_modelling_data[exp_vaep_modelling_data['exp_concedesTrainingSet']]['exp_concedes'].mean(), exp_vaep_modelling_data[exp_vaep_modelling_data['exp_concedesTestSet']]['exp_concedes'].mean(), exp_vaep_modelling_data[exp_vaep_modelling_data['exp_concedesValidationSet']]['exp_concedes'].mean()

(0.16476937483949003, 0.16462848910445474, 0.16308635116629228)

In [21]:
exp_vaep_modelling_data.to_csv("../data/exp_vaep_modelling_data.csv", index=False)